# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Actividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  * A00378771 - Hiram Garcia Austria
  * A01281536 - Joaquín Díaz Hernández
  * A01796568 - Jesús Antonio López Wayas
  * A01795624 - Victor Hugo Vázquez Herrera

* **Número de Equipo: 36**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [50]:
import re
import torch
import librosa

from nltk.corpus import stopwords
from transformers import pipeline
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

nombres_de_archivo = ["01", "04", "05", "06", "14", "22", "24", "25", "26", "27"]
archivos_de_audio = []

for nombre in nombres_de_archivo:
    audio_path = f"./audios/21144-{nombre}.mp3"
    print(f"Cargando archivo: {audio_path}...")
    # Cargar el archivo de audio
    audio, sample_rate = librosa.load(audio_path)
    archivos_de_audio.append((audio, sample_rate))

Cargando archivo: ./audios/21144-01.mp3...
Cargando archivo: ./audios/21144-04.mp3...
Cargando archivo: ./audios/21144-05.mp3...
Cargando archivo: ./audios/21144-06.mp3...
Cargando archivo: ./audios/21144-14.mp3...
Cargando archivo: ./audios/21144-22.mp3...
Cargando archivo: ./audios/21144-24.mp3...
Cargando archivo: ./audios/21144-25.mp3...
Cargando archivo: ./audios/21144-26.mp3...
Cargando archivo: ./audios/21144-27.mp3...


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [8]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

model_id = "openai/whisper-large-v3-turbo" # una versión ligera de v3

# Cargamos el modelo:
processor = AutoProcessor.from_pretrained(model_id)
modelo = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)

# Inicializamos el pipeline:
pipe = pipeline("automatic-speech-recognition",
    model=modelo,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float32,      #torch.float16 if torch.cuda.is_available() else torch.float32,
    device=device
)

Usando dispositivo: cpu


Device set to use cpu


In [9]:
resultados = {}
contador = 0

# Extraemos el texto del audio y lo guardamos en un diccionario:
for audio, sample_rate in archivos_de_audio:
    nombre_de_archivo = f"audio{nombres_de_archivo[contador]}"
    print(f"Procesando {nombre_de_archivo}...")
    result = pipe(audio,
        return_timestamps=True, # Para audios mayores a 30 segs.
        generate_kwargs={"language":"es"} # Lo detecta automático, pero se le puede indicar el idioma.
    )
    resultados[nombre_de_archivo] = result["text"]
    contador += 1

c:\Users\hille\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=es, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=es.


Procesando audio01...


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Procesando audio04...


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Procesando audio05...
Procesando audio06...
Procesando audio14...
Procesando audio22...
Procesando audio24...
Procesando audio25...
Procesando audio26...


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Procesando audio27...


In [15]:
# Mostramos los resultados:
for nombre, texto in resultados.items():
    print(f"{nombre}: {texto}")
    print("-" * 100)

audio01:  Las Fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo Un pequeño corderito decidió refugiarse en un templo cercano Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro Lo inmolaría a su dios Mejor así, replicó el cordero Prefiero ser víctima para un dios A tener que perecer en tus colmillos Si sin remedio vamos a ser sacrificados Más nos vale que sea con el mayor honor Fin de la fábula Esta es una grabación del dominio público
----------------------------------------------------------------------------------------------------
audio04:  Las Fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz Fábula Número 64 El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situac

## **Comentario del por qué seleccionamos este modelo**

El modelo seleccionado es **"openai/whisper-large-v3-turbo"** que es una versión optimizada y ligera del modelo Whisper v3, lo que permite un procesamiento más rápido y eficiente de los audios, manteniendo una alta precisión en la transcripción del habla a texto. Además, es compatible con múltiples idiomas, como el español, y puede manejar audios de diferentes longitudes, lo que lo hace adecuado para esta tarea.

También es importante mencionar que el uso de un modelo optimizado como este permite aprovechar mejor los recursos de hardware disponibles, especialmente en dispositivos con GPU, lo que mejora significativamente el rendimiento en comparación con modelos más pesados.

Por ultimo, cabe mencionar que ademas de las características antes mencionadas, este modelo es gratuito y de código abierto, lo que facilita su utilización constante, sin necesidad de gastar dinero cada vez que lo corremos. Esto lo convierte en una opción ideal para aplicaciones académicas como esta, que requieren transcripción de voz a texto de alta calidad.

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

textos_limpios = []

#Eliminando el texto común de cada uno de los audios:
for nombre, texto in resultados.items():
    texto = texto.lower()  # Convertimos a minúsculas para evitar problemas de coincidencia
    # quitar texto del principio
    texto = re.sub(r'las fábulas .*fábula número', '', texto)
    # quitar texto del final
    texto = re.sub(r'fin de la fábula.*|fin de fábula.*', '', texto)
    textos_limpios.append(texto.strip())

# Mostramos los textos limpios:
for texto in textos_limpios:
    print(texto)
    print("-" * 100)

61 el lobo y el cordero en el templo dándose cuenta de que era perseguido por un lobo un pequeño corderito decidió refugiarse en un templo cercano lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro lo inmolaría a su dios mejor así, replicó el cordero prefiero ser víctima para un dios a tener que perecer en tus colmillos si sin remedio vamos a ser sacrificados más nos vale que sea con el mayor honor
----------------------------------------------------------------------------------------------------
64 el lobo y la coruja a un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. pidió entonces la cancelación de la paga convenida. oye amiga, dijo el lobo, ¿no crees que es suficiente paga 

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

my_stopwords = set(stopwords.words('spanish'))

# Función para tokenizar y limpiar el texto
def tokenizar_y_limpiar(doc):
  # 1. Solo consideramos caracteres letras y acentuadas.
  doc = re.sub(r'[^a-záéíóúüñ]', ' ', doc)
  # 2. Eliminamos espacios extras.
  doc = re.sub(r'\s+', ' ', doc).strip()
  # 3. Tokenizamos el texto, eliminamos stopwords, y conservamos tokens mayores a 1 carácter.
  tokens = [x for x in doc.split() if x not in my_stopwords and len(x) > 1]

  return tokens

# Tokenizamos y limpiamos los textos:
textos_tokenizados = [tokenizar_y_limpiar(texto) for texto in textos_limpios]

# Mostramos los textos tokenizados:
for tokens in textos_tokenizados:
    print(tokens)
    print("-" * 100)

['lobo', 'cordero', 'templo', 'dándose', 'cuenta', 'perseguido', 'lobo', 'pequeño', 'corderito', 'decidió', 'refugiarse', 'templo', 'cercano', 'llamó', 'lobo', 'dijo', 'si', 'sacrificador', 'encontraba', 'allí', 'adentro', 'inmolaría', 'dios', 'mejor', 'así', 'replicó', 'cordero', 'prefiero', 'ser', 'víctima', 'dios', 'tener', 'perecer', 'colmillos', 'si', 'remedio', 'vamos', 'ser', 'sacrificados', 'vale', 'mayor', 'honor']
----------------------------------------------------------------------------------------------------
['lobo', 'coruja', 'lobo', 'comía', 'hueso', 'atragantó', 'hueso', 'garganta', 'corría', 'todas', 'partes', 'busca', 'auxilio', 'encontró', 'correr', 'grulla', 'pidió', 'salvara', 'aquella', 'situación', 'enseguida', 'pagaría', 'ello', 'aceptó', 'grulla', 'introdujo', 'cabeza', 'boca', 'lobo', 'sacando', 'garganta', 'hueso', 'atravesado', 'pidió', 'entonces', 'cancelación', 'paga', 'convenida', 'oye', 'amiga', 'dijo', 'lobo', 'crees', 'suficiente', 'paga', 'haber', '

### **Justificación de los pasos de limpieza utilizados.**

Es necesario limpiar los textos extraídos de los audios para mejorar la calidad del análisis posterior. Los pasos realizados fueron los siguientes:
1. Eliminamos números y signos de puntuación para enfocarnos en las palabras relevantes.
2. Eliminamos espacios extra para evitar problemas de tokenización.
3. Tokenizamos el texto y eliminamos las stopwords para centrarnos en las palabras clave que aportan significado.
4. Conservamos tokens mayores a 1 carácter para evitar palabras irrelevantes.


# **Ejercicio 4:**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**